In [1]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=1372b35e832e68b56f861b76a22d4c8810d02b6853f404ba15f34d8963be8c5b
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *

In [3]:
my_conf = SparkConf()

In [4]:
my_conf.set("My.App.Name", "Date_Practise")
my_conf.set("master", "local[*]")

In [5]:
spark = SparkSession.builder.config(conf=my_conf).getOrCreate()

In [6]:
inputDF = spark.read.csv("/content/biglog.txt",header=True)

In [7]:
inputDF.show()

+-----+-------------------+
|level|           datetime|
+-----+-------------------+
|DEBUG|  2015-2-6 16:24:07|
| WARN| 2016-7-26 18:54:43|
| INFO|2012-10-18 14:35:19|
|DEBUG| 2012-4-26 14:26:50|
|DEBUG| 2013-9-28 20:27:13|
| INFO| 2017-8-20 13:17:27|
| INFO| 2015-4-13 09:28:17|
|DEBUG| 2015-7-17 00:49:27|
|DEBUG| 2014-7-26 02:33:09|
| INFO| 2016-1-13 09:51:57|
|DEBUG| 2015-1-14 08:55:30|
|DEBUG| 2016-1-20 03:47:06|
|DEBUG|  2013-7-8 21:00:50|
|DEBUG| 2012-5-22 11:43:57|
|DEBUG| 2013-3-20 06:14:50|
| INFO|  2015-8-8 20:49:22|
| WARN| 2015-1-14 20:05:00|
| INFO| 2017-6-14 00:08:35|
| INFO| 2016-1-18 11:50:14|
|DEBUG|  2017-7-1 12:55:02|
+-----+-------------------+
only showing top 20 rows



In [8]:
inputDF.createOrReplaceTempView("logging")

In [9]:
spark.sql("select * from logging").show()

+-----+-------------------+
|level|           datetime|
+-----+-------------------+
|DEBUG|  2015-2-6 16:24:07|
| WARN| 2016-7-26 18:54:43|
| INFO|2012-10-18 14:35:19|
|DEBUG| 2012-4-26 14:26:50|
|DEBUG| 2013-9-28 20:27:13|
| INFO| 2017-8-20 13:17:27|
| INFO| 2015-4-13 09:28:17|
|DEBUG| 2015-7-17 00:49:27|
|DEBUG| 2014-7-26 02:33:09|
| INFO| 2016-1-13 09:51:57|
|DEBUG| 2015-1-14 08:55:30|
|DEBUG| 2016-1-20 03:47:06|
|DEBUG|  2013-7-8 21:00:50|
|DEBUG| 2012-5-22 11:43:57|
|DEBUG| 2013-3-20 06:14:50|
| INFO|  2015-8-8 20:49:22|
| WARN| 2015-1-14 20:05:00|
| INFO| 2017-6-14 00:08:35|
| INFO| 2016-1-18 11:50:14|
|DEBUG|  2017-7-1 12:55:02|
+-----+-------------------+
only showing top 20 rows



In [66]:
clubbedDF = spark.sql("select level,date_format(datetime,'MMMM') as Month from logging")

In [67]:
clubbedDF.show()

+-----+---------+
|level|    Month|
+-----+---------+
|DEBUG| February|
| WARN|     July|
| INFO|  October|
|DEBUG|    April|
|DEBUG|September|
| INFO|   August|
| INFO|    April|
|DEBUG|     July|
|DEBUG|     July|
| INFO|  January|
|DEBUG|  January|
|DEBUG|  January|
|DEBUG|     July|
|DEBUG|      May|
|DEBUG|    March|
| INFO|   August|
| WARN|  January|
| INFO|     June|
| INFO|  January|
|DEBUG|     July|
+-----+---------+
only showing top 20 rows



In [15]:
clubbedDF.createOrReplaceTempView("logging_level")

In [17]:
spark.sql("select * from logging_level").show()

+-----+---------+
|level|    Month|
+-----+---------+
|DEBUG| February|
| WARN|     July|
| INFO|  October|
|DEBUG|    April|
|DEBUG|September|
| INFO|   August|
| INFO|    April|
|DEBUG|     July|
|DEBUG|     July|
| INFO|  January|
|DEBUG|  January|
|DEBUG|  January|
|DEBUG|     July|
|DEBUG|      May|
|DEBUG|    March|
| INFO|   August|
| WARN|  January|
| INFO|     June|
| INFO|  January|
|DEBUG|     July|
+-----+---------+
only showing top 20 rows



In [18]:
spark.sql("select level,Month,count(*) from logging_level group by level,month").show()

+-----+---------+--------+
|level|    Month|count(1)|
+-----+---------+--------+
| WARN|     June|    8191|
| INFO|     June|   29143|
|ERROR| November|    3389|
|FATAL|  January|      94|
| WARN| December|    8328|
| WARN|    March|    8165|
|DEBUG|     July|   42085|
|ERROR|    April|    4107|
|ERROR|  January|    4054|
|FATAL|September|      81|
|FATAL|    April|      83|
| INFO|September|   29038|
|FATAL| November|   16797|
|FATAL|  October|      92|
| INFO| February|   28983|
| WARN|    April|    8277|
|DEBUG| December|   41749|
|FATAL| December|      94|
| WARN|      May|    8403|
|ERROR|     June|    4059|
+-----+---------+--------+
only showing top 20 rows



In [15]:
spark.sql("select level,date_format(datetime,'MMMM') as Month, count(*) as count from logging group by level, Month order by Month").show()

+-----+--------+-----+
|level|   Month|count|
+-----+--------+-----+
| WARN|   April| 8277|
|ERROR|   April| 4107|
| INFO|   April|29302|
|FATAL|   April|   83|
|DEBUG|   April|41869|
|FATAL|  August|   80|
| WARN|  August| 8381|
| INFO|  August|28993|
|DEBUG|  August|42147|
|ERROR|  August| 3987|
|DEBUG|December|41749|
|FATAL|December|   94|
|ERROR|December| 4106|
| INFO|December|28874|
| WARN|December| 8328|
| INFO|February|28983|
|ERROR|February| 4013|
|DEBUG|February|41734|
|FATAL|February|   72|
| WARN|February| 8266|
+-----+--------+-----+
only showing top 20 rows



In [16]:
# Here when i do order by it is working but i wanted it as month wise like january,febraury,March...
# how to do it ?
# What if i exact the month number and sort based on the month number does that work?
# How to extract month number from date in sql?

In [17]:
spark.sql("""select level,date_format(datetime,'MMMM') as Month, 
date_format(datetime,'M') as Monthnum, 
count(*) as count 
from logging 
group by level, Month 
order by Month""").show()

AnalysisException: ignored

In [19]:
spark.sql("""select level,date_format(datetime,'MMMM') as Month, 
first(date_format(datetime,'M')) as Monthnum, 
count(*) as count 
from logging 
group by level, Month 
order by Monthnum""").show()

+-----+--------+--------+-----+
|level|   Month|Monthnum|count|
+-----+--------+--------+-----+
|FATAL| January|       1|   94|
|DEBUG| January|       1|41961|
| INFO| January|       1|29119|
|ERROR| January|       1| 4054|
| WARN| January|       1| 8217|
|DEBUG| October|      10|41936|
| WARN| October|      10| 8226|
|ERROR| October|      10| 4040|
| INFO| October|      10|29018|
|FATAL| October|      10|   92|
|DEBUG|November|      11|33366|
|FATAL|November|      11|16797|
| INFO|November|      11|23301|
| WARN|November|      11| 6616|
|ERROR|November|      11| 3389|
|FATAL|December|      12|   94|
|DEBUG|December|      12|41749|
|ERROR|December|      12| 4106|
| INFO|December|      12|28874|
| WARN|December|      12| 8328|
+-----+--------+--------+-----+
only showing top 20 rows



In [22]:
# Here i am getting the january but still it is not the right output but why?
# is our monthnum is string or integer?
# lets try converting the monthnum to integer
# so how to convert a string to integer in sql : sol: we can use cast fucntion

In [58]:
spark.sql("""select level,date_format(datetime,'MMMM') as Month, 
cast(first(date_format(datetime,'M')) as int) as Monthnum, 
count(*) as count 
from logging 
group by level, Month 
order by Monthnum""").show()

+-----+--------+--------+-----+
|level|   Month|Monthnum|count|
+-----+--------+--------+-----+
|FATAL| January|       1|   94|
|DEBUG| January|       1|41961|
| INFO| January|       1|29119|
|ERROR| January|       1| 4054|
| WARN| January|       1| 8217|
|DEBUG|February|       2|41734|
| WARN|February|       2| 8266|
|ERROR|February|       2| 4013|
| INFO|February|       2|28983|
|FATAL|February|       2|   72|
|ERROR|   March|       3| 4122|
|FATAL|   March|       3|   70|
|DEBUG|   March|       3|41652|
| WARN|   March|       3| 8165|
| INFO|   March|       3|29095|
|FATAL|   April|       4|   83|
|DEBUG|   April|       4|41869|
|ERROR|   April|       4| 4107|
| INFO|   April|       4|29302|
| WARN|   April|       4| 8277|
+-----+--------+--------+-----+
only showing top 20 rows



In [59]:
# So now i am getting the output i want in the month column but if you level it is not ordered lets order that to..
spark.sql("""select level,date_format(datetime,'MMMM') as Month, 
cast(first(date_format(datetime,'M')) as int) as Monthnum, 
count(*) as count 
from logging 
group by level, Month 
order by Monthnum, level""").show()

+-----+--------+--------+-----+
|level|   Month|Monthnum|count|
+-----+--------+--------+-----+
|DEBUG| January|       1|41961|
|ERROR| January|       1| 4054|
|FATAL| January|       1|   94|
| INFO| January|       1|29119|
| WARN| January|       1| 8217|
|DEBUG|February|       2|41734|
|ERROR|February|       2| 4013|
|FATAL|February|       2|   72|
| INFO|February|       2|28983|
| WARN|February|       2| 8266|
|DEBUG|   March|       3|41652|
|ERROR|   March|       3| 4122|
|FATAL|   March|       3|   70|
| INFO|   March|       3|29095|
| WARN|   March|       3| 8165|
|DEBUG|   April|       4|41869|
|ERROR|   April|       4| 4107|
|FATAL|   April|       4|   83|
| INFO|   April|       4|29302|
| WARN|   April|       4| 8277|
+-----+--------+--------+-----+
only showing top 20 rows



In [33]:
# So now it is okay makes some sensable compare to previous one. But, there 
#is one extra column rite which is monthnum i dont want it lets drop it.
# how to drop it just use drop transformation.
# lets store it to a dataframe so that we could use drop.

In [60]:
result = spark.sql("""select level,date_format(datetime,'MMMM') as Month, 
cast(first(date_format(datetime,'M')) as int) as Monthnum, 
count(*) as count 
from logging 
group by level, Month 
order by Monthnum, level""")

In [39]:
result2 = result.drop("Monthnum")

In [40]:
result2.show(40)

+-----+--------+-----+
|level|   Month|count|
+-----+--------+-----+
|DEBUG| January|41961|
|ERROR| January| 4054|
|FATAL| January|   94|
| INFO| January|29119|
| WARN| January| 8217|
|DEBUG|February|41734|
|ERROR|February| 4013|
|FATAL|February|   72|
| INFO|February|28983|
| WARN|February| 8266|
|DEBUG|   March|41652|
|ERROR|   March| 4122|
|FATAL|   March|   70|
| INFO|   March|29095|
| WARN|   March| 8165|
|DEBUG|   April|41869|
|ERROR|   April| 4107|
|FATAL|   April|   83|
| INFO|   April|29302|
| WARN|   April| 8277|
|DEBUG|     May|41785|
|ERROR|     May| 4086|
|FATAL|     May|   60|
| INFO|     May|28900|
| WARN|     May| 8403|
|DEBUG|    June|41774|
|ERROR|    June| 4059|
|FATAL|    June|   78|
| INFO|    June|29143|
| WARN|    June| 8191|
|DEBUG|    July|42085|
|ERROR|    July| 3976|
|FATAL|    July|   98|
| INFO|    July|29300|
| WARN|    July| 8222|
|DEBUG|  August|42147|
|ERROR|  August| 3987|
|FATAL|  August|   80|
| INFO|  August|28993|
| WARN|  August| 8381|
+-----+----

In [61]:
# Now it looks good. But make it more sensable because here i am seeing so many fatal etc...all the so many times
#what if i can get it just as one like fatal in january and total
# Here i want all the jan,feb,marc as columns then i can keep only one repitation of fatal, info and all. like tabular format you can imagine.
# So, lets get it done then. how to make rows values to columns then ?
# Apache spark provides something called pivot we could use that and get it. Lets do it.

In [69]:
result = spark.sql("""select level,date_format(datetime,'MMMM') as Month, 
cast(date_format(datetime,'M') as int) as Monthnum 
from logging""").groupBy("level").pivot("Monthnum").count().show(100)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|level|    1|    2|    3|    4|    5|    6|    7|    8|    9|   10|   11|   12|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| INFO|29119|28983|29095|29302|28900|29143|29300|28993|29038|29018|23301|28874|
|ERROR| 4054| 4013| 4122| 4107| 4086| 4059| 3976| 3987| 4161| 4040| 3389| 4106|
| WARN| 8217| 8266| 8165| 8277| 8403| 8191| 8222| 8381| 8352| 8226| 6616| 8328|
|FATAL|   94|   72|   70|   83|   60|   78|   98|   80|   81|   92|16797|   94|
|DEBUG|41961|41734|41652|41869|41785|41774|42085|42147|41433|41936|33366|41749|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+



In [63]:
# SO now it is sensable right and it is easy to see..what is what. 
# we can still do some optimazation to avoid system to calculate the distinct monthnum operation.
# lets do it
# here we have monthnames and which are only 12 months it wont be changed so we could hardcode the month names.

In [71]:
month_column = ['January','February','March','April','May','June','July','August','September','October','November','December']

In [73]:
result = spark.sql("""select level, date_format(datetime,'MMMM') as Month 
from logging""").groupBy("level").pivot("Month",month_column).count().show(100)

+-----+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+
|level|January|February|March|April|  May| June| July|August|September|October|November|December|
+-----+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+
| INFO|  29119|   28983|29095|29302|28900|29143|29300| 28993|    29038|  29018|   23301|   28874|
|ERROR|   4054|    4013| 4122| 4107| 4086| 4059| 3976|  3987|     4161|   4040|    3389|    4106|
| WARN|   8217|    8266| 8165| 8277| 8403| 8191| 8222|  8381|     8352|   8226|    6616|    8328|
|FATAL|     94|      72|   70|   83|   60|   78|   98|    80|       81|     92|   16797|      94|
|DEBUG|  41961|   41734|41652|41869|41785|41774|42085| 42147|    41433|  41936|   33366|   41749|
+-----+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+



In [74]:
# This Looks Good! Good Job we could use this to visulize or analyze the why something occured on somemonth 
# like fatal why there are more FATAL in the month of November and analyse.